This notebook uses an R kernel.

# Sequence Fetch

Author: Zhongyi (James) Guo <br>
Date: 10/29/2024

## Import Packages

In [ ]:
getwd()

In [ ]:
.libPaths()

In [ ]:
library(tidyverse)
library(biomaRt)
library(httr)
library(jsonlite)
library(xml2)
library(stringi)

In [ ]:
mart <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")

## Import Data

### All Genes

In [ ]:
count_clean <- read_tsv('../../result/deseq2//count_clean.tsv')
head(count_clean)

In [ ]:
gene_info <- getBM(
    attributes = c("ensembl_gene_id", "hgnc_symbol", "chromosome_name", 
                   "start_position", "end_position", "strand"),
    filters = "ensembl_gene_id", 
    values = count_clean$NAME, 
    mart = mart
)
head(gene_info)

In [ ]:
gene_info <- gene_info |> 
    mutate(
        upstream_start = ifelse(strand == 1, start_position - 2001, end_position + 2001),
        upstream_end = ifelse(strand == 1, start_position - 1, end_position + 1),
        temp_upstream_start = upstream_start,
        temp_upstream_end = upstream_end) |>
    mutate(
        upstream_start = ifelse(strand == -1, temp_upstream_end, temp_upstream_start),
        upstream_end = ifelse(strand == -1, temp_upstream_start, temp_upstream_end)) |>
    dplyr::select(-c(temp_upstream_start, temp_upstream_end))

In [ ]:
head(gene_info)

In [ ]:
dim(gene_info)

In [ ]:
dim(count_clean)

565238 genes had chromosome coordinates, compared to 581749 total genes present.

Sequence fetching inspired by a post here: https://www.biostars.org/p/354170/

In [ ]:
fetch_sequence <- function(chr, start, end, strand) {
    server <- "http://grch37.rest.ensembl.org"
    ext <- paste0("/sequence/region/human/", chr, ":", start, "..", end, ":", strand)
    r <- GET(paste0(server, ext), content_type("text/plain"))
    sequence <- content(r)
    
    # get the reverse complement if on the negative strand
    if (strand == -1) {
        sequence <- chartr(old="ATGC", new="TACG", sequence) |> stri_reverse()
    }
    
    return(sequence)
}

In [ ]:
system.time({
  gene_info <- gene_info %>%
    mutate(upstream_region = pmap_chr(
      list(chromosome_name, upstream_start, upstream_end, strand), 
      fetch_sequence
    ))
})

In [ ]:
head(gene_info)

In [ ]:
write_tsv(gene_info, '../../result/deseq2/all_gene_sequence.tsv')

### Significant Genes

In [ ]:
sig_gene <- read_csv('../../result/deseq2/sig_gene.csv')
head(sig_gene)

In [ ]:
# only keep the gene names
sig_gene <- sig_gene[, 1]

In [ ]:
sig_gene_info <- getBM(
    attributes = c("ensembl_gene_id", "hgnc_symbol", "chromosome_name", "start_position", "end_position", "strand"),
    filters = "ensembl_gene_id", 
    values = sig_gene$gene, 
    mart = mart
)
head(sig_gene_info)

In [ ]:
sig_gene_info <- sig_gene_info |> 
    mutate(
        upstream_start = ifelse(strand == 1, start_position - 2001, end_position + 2001),
        upstream_end = ifelse(strand == 1, start_position - 1, end_position + 1),
        temp_upstream_start = upstream_start,
        temp_upstream_end = upstream_end) |>
    mutate(
        upstream_start = ifelse(strand == -1, temp_upstream_end, temp_upstream_start),
        upstream_end = ifelse(strand == -1, temp_upstream_start, temp_upstream_end)) |>
    dplyr::select(-c(temp_upstream_start, temp_upstream_end))

In [ ]:
head(sig_gene_info)

In [ ]:
dim(sig_gene_info)

In [ ]:
dim(sig_gene)

8195 significant genes had chromosome coordinates, compared to 8300 total significant genes present.

In [ ]:
system.time({
    sig_gene_info <- sig_gene_info %>%
        mutate(upstream_region <-  pmap_chr(
          list(chromosome_name, upstream_start, upstream_end, strand), 
          fetch_sequence
      ))
    })

In [ ]:
write_tsv(sig_gene_info, '../../result/deseq2/sig_gene_sequence.tsv')

## Conclusion

We extracted upstream 2000 bp sequences for all genes in the RNA-seq count matrix and significant genes identified by DESeq2.

In [ ]:
sessionInfo()